# nanoPU
## 概括
- nanoPU: 一种新的NIC-CPU协同设计，用于加速日益普及的数据中心应用：那些使用许多小型远程过程调用（RPC）且处理时间非常短（μs级）的应用。
- nanoPU最新的aspect: 设计网络和应用程序之间的快速路径，绕过缓存和内存层次结构，将到达的消息直接放入CPU寄存器堆（register file）。
- 快速路径: 包含用于低延迟传输和拥塞控制的可编程硬件支持，以及用于RPCs到核心高效负载平衡的硬件支持。
- 硬件加速的线程调度器可以做出亚纳秒级的决策，从而提高CPU利用率，降低RPC的尾部响应时间。
- 怎么做: 通过修改开源RISC-V CPU构建了一个nanoPU快速路径的FPGA原型，并使用AWS FPGA上的周期精确模拟评估了其性能。
- 结果: 通过nanoPU的线对线RPC响应时间仅为69ns，比同类最佳、低延迟的商用NIC快一个数量级。我们证明，硬件线程调度程序能够将RPC尾部响应时间减少约5倍，同时使系统能够承受比传统线程调度技术高20%的负载。
- 一套评估用的程序: 包括用于文档检索的MICA、Raft和Set Algebra；我们证明了nanoPU可以作为高性能、可编程的替代品用于单边RDMAO操作。

## 介绍
- 现状： 如今，大型在线服务通常部署为软件的多个部分在数据中心运行。层之间使用不同大小和复杂度的远程过程调用（RPC）进行通信。
- 一些RPC调用微服务(microservices)持续数毫秒，而另一些RPC调用远程（无服务器）函数，或检索单个数据段，持续时间仅为数微秒。
- 展望：因此，服务时间为微秒（甚至可能是纳秒）的短信息在未来的数据中心中变得更加普遍
- 利用实例：Facebook数据中心中，交流的消息中有很大一部分是针对单个键值内存引用，越来越多的论文的描述细粒度（通常是缓存驻留）计算基于非常小的RPCs。
- 评估RPC系统性能时，有三个主要指标非常有用：
  - themedian response time 主题响应时间（即，从客户端发出RPC请求到收到响应的时间）用于调用多个连续RPC的应用程序）
  - thetail response time 尾部响应时间 （即，最长或第99%的RPC响应时间）用于具有大扇出的应用程序（例如，mapreduce作业），因为它们必须等待所有RPC完成后才能继续
    - MapReduce是一种编程模型，用于大规模数据集（大于1TB）的并行运算。概念"Map（映射）"和"Reduce（归约）"，是它们的主要思想，都是从函数式编程语言里借来的，还有从矢量编程语言里借来的特性。它极大地方便了编程人员在不会分布式并行编程的情况下，将自己的程序运行在分布式系统上。 当前的软件实现是指定一个Map（映射）函数，用来把一组键值对映射成一组新的键值对，指定并发的Reduce（归约）函数，用来保证所有映射的键值对中的每一个共享相同的键组。
  - thecommunication overhead 通信开销 （即通信与计算的比率）当通信开销很大时，可能根本不值得向远程CPU发送请求。我们有时需要处理管道部分的更具体指标，例如主题线对线延迟，即RPC请求的第一个字节到达NIC服务器直到响应的最后一个字节离开的时间。
- 